In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing as pp
%pylab inline

cause = pd.read_csv('/home/lara/Documents/Repository/Capstone-1_WorldBank_GenderData/causes.csv')
effect = pd.read_csv('/home/lara/Documents/Repository/Capstone-1_WorldBank_GenderData/effects.csv')

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#Supervised Learning Regressor algorithms
from sklearn.linear_model import SGDRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


Populating the interactive namespace from numpy and matplotlib


## Loop through various models

In [2]:
models = []
models.append(('SGDR', SGDRegressor()))
models.append(('GaussianPR', GaussianProcessRegressor()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('DTree', DecisionTreeRegressor()))
models.append(('GradientBR', GradientBoostingRegressor()))
models.append(('SVR', SVR()))
models.append(('RF', RandomForestRegressor(n_jobs = -1, n_estimators = 100)))

In [3]:
#a function to evaluate each model
def run_models(x,y):
    results = []
    names = []

    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
        cv_results = model_selection.cross_val_score(model, x, y, cv=kfold, scoring='neg_mean_squared_error')
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
'''
    # boxplot algorithm comparison
    fig = pyplot.figure()
    fig.suptitle(title)
    ax = fig.add_subplot(111)
    pyplot.boxplot(results)
    ax.set_xticklabels(names)
    pyplot.ylim(0,1)
    pyplot.show()
'''

'\n    # boxplot algorithm comparison\n    fig = pyplot.figure()\n    fig.suptitle(title)\n    ax = fig.add_subplot(111)\n    pyplot.boxplot(results)\n    ax.set_xticklabels(names)\n    pyplot.ylim(0,1)\n    pyplot.show()\n'

# Functions for each model

In [4]:
def run_svr(parameters, x, y):
    kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
    svr = SVR()
    reg = GridSearchCV(svr, parameters, n_jobs =-1, cv = kfold, scoring='neg_mean_squared_error')
    reg.fit(x, y)
    
    print reg.best_estimator_
    print reg.best_score_

In [5]:
def run_knn(parameters, x, y):
    kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
    gpr = KNeighborsRegressor()
    reg = GridSearchCV(gpr, parameters, n_jobs =-1, cv = kfold, scoring='neg_mean_squared_error')
    reg.fit(x, y)
    
    print reg.best_estimator_
    print reg.best_score_

In [6]:
def rand_gradi(parameters, x, y):
    kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
    gbr = GradientBoostingRegressor()
    reg = RandomizedSearchCV(gbr, parameters, n_jobs =-1, cv = kfold, scoring='neg_mean_squared_error', n_iter=10, random_state=5)
    reg.fit(x, y)
    
    print reg.best_estimator_
    print reg.best_score_

In [7]:
def grid_gradi(parameters, x, y):
    kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
    gbr = GradientBoostingRegressor()
    reg = GridSearchCV(gbr, parameters, n_jobs =-1, cv = kfold, scoring='neg_mean_squared_error')
    reg.fit(x, y)
    
    print reg.best_estimator_
    print reg.best_score_

In [8]:
def rand_rf(parameters, x, y):
    kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
    rf = RandomForestRegressor()
    reg = RandomizedSearchCV(rf, parameters, n_jobs =-1, cv = kfold, scoring='neg_mean_squared_error',  n_iter=10, random_state=5)
    reg.fit(x, y)
    
    print reg.best_estimator_
    print reg.best_score_

In [9]:
def grid_rf(parameters, x, y):
    kfold = model_selection.KFold(n_splits=10, shuffle = True, random_state=11)
    rf = RandomForestRegressor()
    reg = GridSearchCV(rf, parameters, n_jobs =-1, cv = kfold, scoring='neg_mean_squared_error')
    reg.fit(x, y)
    
    print reg.best_estimator_
    print reg.best_score_

# Contraceptive Use as the Dependent Variable
### Independent Variables: GDP, Spending on Health, Spending on Education

In [69]:
#break "cause" data into training and testing sets
xcause, xCtest, ycause, yCtest = train_test_split(cause[['gdp','healthspend', 'eduspend']].values, cause.bc.values, random_state = 10)

In [5]:
# use "cause" training sets to evaluate model fitting
run_models(xcause, ycause)

SGDR: -106480394964309420429719382127542272.000000 (242724671690324759514818676927758336.000000)
GaussianPR: -2730.725384 (247.353779)
KNN: -394.511996 (49.295327)
DTree: -604.915170 (121.014826)
GradientBR: -354.821604 (38.685420)
SVR: -549.321570 (73.052767)
RF: -336.736606 (38.564862)


#### KNN, SVR, Gradient Boosting Regressor and Random Forest appear to be the best performing models
## Tuning the models:

### SVM

In [70]:
param_grid = {'C': [0.001, 0.1, 1, 10,50, 100, 150, 500,750,900, 1000,1100,1250, 1500], 'epsilon': [0.00001, 0.0001, 0.001, 0.1, 1, 5, 8,10,15,20, 50, 100], 'gamma': [1e-9, 1e-8, 5e-8,7e-8, 1e-7, 5e-7, 1e-6, 1e-5, .0001, .001, .01, .1, 1, 10]}
run_svr(param_grid, xcause, ycause)

SVR(C=1250, cache_size=200, coef0=0.0, degree=3, epsilon=10, gamma=7e-08,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
-340.73986383


In [45]:
#Validate with test set data (xCtest, yCtest)
svr_final = SVR(C=1250, cache_size=200, coef0=0.0, degree=3, epsilon=10, gamma=7e-08,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

svr_final.fit(xcause, ycause)
mean_squared_error(svr_final.predict(xCtest), yCtest)

341.66895418756087

### KNN

In [11]:
param_knn = { 'n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 'weights':['uniform', 'distance'], 'leaf_size': [1,5,10,15,20,25,30,35,40,45,50]}
run_knn(param_knn, xcause, ycause)

KNeighborsRegressor(algorithm='auto', leaf_size=1, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=19, p=2,
          weights='uniform')
-353.937164213


In [16]:
#Validate with testing data
knn_final = KNeighborsRegressor(algorithm='auto', leaf_size=1, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=19, p=2,
          weights='uniform')
knn_final.fit(xcause, ycause)
mean_squared_error(knn_final.predict(xCtest), yCtest)

349.00778679808928

### Gradient Boosting Regressor

In [22]:
param_gbr = {'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450], 'max_depth':[1,2,3,4,5,10,15,20,25,50,100,150, None]}
rand_gradi(param_gbr, xcause, ycause)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
-345.282333241


In [15]:
paramgrid_gbr = {'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450], 'max_depth':[1,2,3,4,5,10,15,20,25,50,100,150]}
grid_gradi(paramgrid_gbr, xcause, ycause)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
-335.98372412


In [14]:
# Vaildate Model with test data, xCtest, yCtest
gbr_final =GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
gbr_final.fit(xcause, ycause)
mean_squared_error(gbr_final.predict(xCtest),yCtest)

327.98620879046996

### Random Forest

In [13]:
param_dist_rf = {'n_estimators': [10, 100, 250, 500, 750, 1000, 1250, 1500], 'max_depth': [1,3,4,5,6,7,8,9,10,20,50,100]}
rand_rf(param_dist_rf, xcause,ycause)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=750, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-329.287565245


In [71]:
#Full Grid Search CV
paramgrid_rf = {'n_estimators': [5, 10, 50, 75, 100, 150, 250, 500, 750, 1000, 1250, 1500], 'max_depth': [1,2,3,4,5,6,7,8,9,10,20,25,50,75,100, None], 'n_jobs': [-1]}
grid_rf(paramgrid_rf, xcause,ycause)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=150, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-326.851922116


In [72]:
# Validate with test data:  xCtest, yCtest
rf_final= RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=150, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
rf_final.fit(xcause, ycause)
mean_squared_error(rf_final.predict(xCtest),yCtest)

323.2377311606383

# Contraceptive Use as the Independent Variable

## Life Expectancy

In [11]:
# Make train and test sets out of Contraceptive Use and Life Expectancy
xlife, xLtest, ylife, yLtest = train_test_split(effect[['life']].values, effect.bc.values, random_state = 10)

In [31]:
# use "effect" training set to evaluate model fitting
run_models(xlife, ylife)

SGDR: -16738543191602606083407872.000000 (7287401765340772013441024.000000)
GaussianPR: -1568351.678445 (4650492.489114)
KNN: -232.139862 (53.909149)
DTree: -428.298147 (94.433701)
GradientBR: -229.623654 (52.544546)
SVR: -272.038214 (43.263374)
RF: -302.099889 (71.252258)


### SVR

In [12]:
param_life = {'C': [0.001, 0.1, 0.5,1,2, 3,4,5,10,15,20,50, 100, 1000], 'epsilon': [0.00001, 0.0001, 0.001, 0.1, 1,5,6,7,8,9,10,15,20,50,100], 'gamma': [1e-6, 1e-5, .0001, .001, .005,.008,.01,.015,.05, .1, 1, 10]}
run_svr(param_life, xlife,ylife)

SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=6, gamma=0.005,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
-193.014404395


In [9]:
#validate with test data
svr_life = SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=6, gamma=0.005,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr_life.fit(xlife, ylife)
mean_squared_error(svr_life.predict(xLtest), yLtest)

225.27613804288893

### KNN

In [7]:
param_knn = { 'n_neighbors': [1,2,3,4,5,6,7,8,9,10,15,20], 'weights':['uniform', 'distance']}
run_knn(param_knn, xlife, ylife)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=20, p=2,
          weights='uniform')
-201.51451708


In [12]:
param_knn = { 'n_neighbors': [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45], 'weights':['uniform', 'distance']}
run_knn(param_knn, xlife, ylife)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=40, p=2,
          weights='uniform')
-191.663243999


In [13]:
#validate with test data
knn_life = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=40, p=2,
          weights='uniform')
knn_life.fit(xlife, ylife)
mean_squared_error(knn_life.predict(xLtest), yLtest)

222.83196311124061

### Gradient Boosting Regressor

In [15]:
param_gbr = {'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450], 'max_depth':[1,2,3,4,5,10,15,20,25,50,100,150, None]}
rand_gradi(param_gbr, xlife, ylife)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
-208.571582313


In [18]:
param_gbr = {'n_estimators': [10, 25,50,75,100,125, 150, 200], 'max_depth':[1,2,3,4,5,10,15,20, None]}
rand_gradi(param_gbr, xlife, ylife)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
-200.151770331


In [ ]:
#Full Gridsearch
grid_gradi(param_gbr, xlife, ylife)

In [19]:
#validate with test data
gbr_life = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
gbr_life.fit(xlife, ylife)
mean_squared_error(gbr_life.predict(xLtest), yLtest)

233.81826409215049

### Random Forest Regressor

In [29]:
param_dist_rf = {'n_estimators': [25,50,100,200,250,300,400,500,600,750,900, 1000, 1250, 1500, 1750], 'max_depth': [1,3,4,5,6,7,8,9,10,20,50,100, None]}
rand_rf(param_dist_rf, xlife,ylife)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-197.037953371


In [8]:
#Full Grid CV
grid_rf(param_dist_rf, xlife,ylife)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-196.581474044


In [13]:
#Validate with test data
rf_life = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
rf_life.fit(xlife, ylife)
mean_squared_error(rf_life.predict(xLtest), yLtest)

239.79020854702082

## Maternal Death Rate

In [11]:
# Make train and test sets out of Contraceptive Use and Maternal Death Rate
xmat, xMtest, ymat, yMtest = train_test_split(effect[['matdeath']].values, effect.bc.values, random_state = 10)

In [16]:
# use "effect" training set to evaluate model fitting
run_models(xmat, ymat)

SGDR: -249862009102755255725559119872.000000 (604487680873706000823569350656.000000)
GaussianPR: -575.991368 (83.749059)
KNN: -250.437533 (54.483379)
DTree: -350.080495 (69.626539)
GradientBR: -232.837089 (52.417803)
SVR: -493.738410 (55.055903)
RF: -276.079008 (58.179668)


### SVR

In [19]:
param_mat = {'C': [0.001, 0.1,1,5,8,10,15,30,40,50,60,70,80,90,100,110,120,130,140,150,500,1000], 'epsilon': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1,5,10,13,15,17,20,30,50,100], 'gamma': [1e-7,5e-7,1e-6,2e-5,5e-6, 1e-5,3e-5, 5e-5, .0001, .001, .01, .1, 1, 10]}
run_svr(param_mat, xmat,ymat)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=13, gamma=5e-07,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
-223.489027708


In [21]:
#validate with test data
svr_mat = SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=13, gamma=5e-07,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr_mat.fit(xmat, ymat)
mean_squared_error(svr_mat.predict(xMtest), yMtest)

220.41213896719375

### KNN

In [18]:
paramknn_mat = { 'n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}
run_knn(paramknn_mat, xmat, ymat)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=13, p=2,
          weights='uniform')
-220.684355972


In [22]:
#validate with test data
knn_mat = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=13, p=2,
          weights='uniform')
knn_mat.fit(xmat, ymat)
mean_squared_error(knn_mat.predict(xMtest), yMtest)

225.74045468850721

### Gradient Boosting Regressor

In [26]:
param_gbr_mat = {'loss':['ls', 'lad', 'huber', 'quantile'], 'max_depth':[1,3,5,10,20,30,50,75,100], 'min_samples_leaf':[1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15, 20,25, 50, 100],'n_estimators': [10, 100,400,450,500,550,600,750, 1000]}
rand_gradi(param_gbr_mat, xmat, ymat)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=2, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=600,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
-238.676329002


In [27]:
#Full Gridsearch
grid_gradi(param_gbr_mat, xmat,ymat)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=20, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
-221.74463557


In [28]:
#validate with test data
gbr_mat = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=20, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
gbr_mat.fit(xmat, ymat)
mean_squared_error(gbr_mat.predict(xMtest), yMtest)

221.36404730871104

### Random Forest

In [37]:
param_rf_mat = {'n_estimators': [10, 100, 250, 500,600, 750, 900,1000, 1250, 1500, 1750, 2000], 'max_depth': [1,3,4,5,6,7,8,9,10,20,50,100]}
rand_rf(param_rf_mat, xmat, ymat)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=600, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-226.364560745


In [38]:
#Full Gridsearch CV
grid_rf(param_rf_mat, xmat, ymat)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-223.601777109


In [39]:
#validate with test data
rf_mat = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
rf_mat.fit(xmat, ymat)
mean_squared_error(rf_mat.predict(xMtest), yMtest)

234.30917193256718

## Teen Fertility Rate

In [41]:
# Make train and test sets out of Contraceptive Use and Teen Pregnancy Rate
xteen, xTtest, yteen, yTtest = train_test_split(effect[['teen']].values, effect.bc.values, random_state = 10)

In [19]:
# use "effect" training set to evaluate model fitting
run_models(xteen, yteen)

SGDR: -20025273435024791347134464.000000 (35556532195793555781844992.000000)
GaussianPR: -90670170.251689 (125975190.459412)
KNN: -370.315242 (66.435124)
DTree: -578.382639 (96.034763)
GradientBR: -367.394936 (78.407684)
SVR: -483.662031 (54.238509)
RF: -435.076509 (87.235137)


### SVR

In [45]:
param_teen = {'C': [0.001, 0.1,0.5,1,2,3,4,5,10,50,100,500,1000], 'epsilon': [0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1,2,3,4,5,6,7,8,9,10,50,100], 'gamma': [1e-7,1e-6,1e-5,5e-5, .0001,.0005, .001, .01, .1, 1, 10]}
run_svr(param_teen, xteen,yteen)

SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=7, gamma=0.0001,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
-341.721172173


In [46]:
#validate with test data
svr_teen = SVR(C=2, cache_size=200, coef0=0.0, degree=3, epsilon=7, gamma=0.0001,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr_teen.fit(xteen, yteen)
mean_squared_error(svr_teen.predict(xTtest), yTtest)

334.95649070311248

### KNN

In [49]:
paramknn_teen = { 'n_neighbors': [1,2,3,4,5,6,7,8,9,10,15,16,17,18,19,20,21,22,23,24,25]}
run_knn(paramknn_teen, xteen, yteen)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=23, p=2,
          weights='uniform')
-328.929050825


In [50]:
#validate with test data
svr_teen = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=23, p=2,
          weights='uniform')
svr_teen.fit(xteen, yteen)
mean_squared_error(svr_teen.predict(xTtest), yTtest)

347.97135808612069

### Gradient Boosting Regressor

In [61]:
param_gbr_teen = {'loss':['ls', 'lad', 'huber', 'quantile'], 'n_estimators': [5, 10, 20,50,75,100,250,400,500,600,700, 1000], 'max_depth':[1,2,3,4,5,8,9,10,15,20,50,100,150], 'min_samples_leaf':[1, 2, 3, 4, 5,8,9, 10,15,20, 25, 50,75,100, 125]}
rand_gradi(param_gbr_teen, xteen, yteen)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=20, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)
-340.854730282


In [62]:
#Full Grid CV
grid_gradi(param_gbr_teen, xteen, yteen)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=15, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
-330.003679829


In [63]:
#validate with test data
gbr_teen = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=15, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=50, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)
gbr_teen.fit(xteen, yteen)
mean_squared_error(gbr_teen.predict(xTtest), yTtest)

332.82153872505177

### Random Forest

In [65]:
param_rf_teen = {'n_estimators': [10, 100, 250, 500, 600,700,800,900, 1000, 1250,1300, 1500, 1700, 2000], 'max_depth': [1,3,4,5,6,7,8,9,10,20,50,100]}
rand_rf(param_rf_teen, xteen, yteen)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=900, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-335.725744463


In [66]:
#Full Grid CV
grid_rf(param_rf_teen, xteen, yteen)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=900, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
-333.427243338


In [67]:
#validate with test data
rf_teen = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=900, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
rf_teen.fit(xteen, yteen)
mean_squared_error(rf_teen.predict(xTtest), yTtest)

316.54408636943674